# EDA

In [107]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
import pandas as pd
dtype_spec = {6: str, 7: str, 13: str, 14: str, 27: str, 49: str, 69: str, 71: str, 74: str, 75: str}
data = pd.read_csv('../data/incidents.csv', low_memory=False)

In [ ]:
# data.head()
# data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28172 entries, 0 to 28171
Data columns (total 70 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Credit Score Bucket v2                                 28172 non-null  object 
 1   Age Bucket v2                                          28172 non-null  object 
 2   Market Name                                            28166 non-null  object 
 3   Creation Date (ET)                                     28172 non-null  object 
 4   Actual Pickup Date (ET)                                9740 non-null   object 
 5   Returned Date (ET)                                     6723 non-null   object 
 6   Days With Vehicle                                      9740 non-null   float64
 7   Miles Driven                                           9825 non-null   float64
 8   Scheduled Pickup Date (ET)                    

In [28]:
# data['Single Party Incidents'] = data['Single Party Incidents'].fillna(0)
# data['Multi Party Incidents'] = data['Multi Party Incidents'].fillna(0)

In [48]:
# drop ingle Party Incidents and Multi Party Incidents 
data = data.drop(columns=['Single Party Incidents', 'Multi Party Incidents', 'early_selfchurn_customer_no_accdient', 
                          'seasoning_3month_SinglePartyIncident', 'seasoning_3month_MultiPartyIncident',
                          'Legacy Order Id', 'Most Recent Vehicle Id', 'Order ID', 'Subscriber ID'])
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9740 entries, 0 to 9739
Data columns (total 74 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Credit Score Bucket v2                                 9740 non-null   object 
 1   Age Bucket v2                                          9740 non-null   object 
 2   Market Name                                            9740 non-null   object 
 3   Creation Date (ET)                                     9740 non-null   object 
 4   Actual Pickup Date (ET)                                9740 non-null   object 
 5   Returned Date (ET)                                     6721 non-null   object 
 6   Days With Vehicle                                      9740 non-null   float64
 7   Miles Driven                                           9689 non-null   float64
 8   Scheduled Pickup Date (ET)                      

In [49]:
target = 'incident_Ind'
data[target].mean()

np.float64(0.07279260780287475)

In [51]:
missing_ratio = data.isnull().mean()
missing_ratio[missing_ratio>0.2].sort_values()

Initial Vehicle Product Group Short        0.207598
Initial Vehicle Product Group              0.207598
Most Recent Vehicle Product Group          0.230185
Most Recent Vehicle Product Group Short    0.230185
Returned Date (ET)                         0.309959
Mileage Plan at Return                     0.310062
Total Unused Mile Credits at Return        0.321355
Insurance Fee at Order                     0.325051
Insurance Plan at Order                    0.325051
Mileage Plan at Return Sorted              0.342197
first_multiparty                           0.901232
days_in_car_when_MP                        0.901232
Repo Process Started Date                  0.932238
first_singleparty                          0.966632
days_in_car_when_SP                        0.966632
Cancellation Date (ET)                     1.000000
dtype: float64

# Modeling

In [243]:
import pandas as pd
import numpy as np
data = pd.read_csv('../data/incidents.csv', low_memory=False)
data['Miles Driven'] = data['Miles Driven'].fillna(0)
data['Insurance Fee at Order'] = data['Insurance Fee at Order'].fillna(0)
data['Total Unused Mile Credits at Return'] = data['Total Unused Mile Credits at Return'].fillna(0)

In [244]:
# lable the missing data
missing_ratio = data.isnull().mean()
cols_missing_value = missing_ratio[missing_ratio>0].index
df_missing_flag = pd.DataFrame()
for col in cols_missing_value:
    df_missing_flag[col+'_missing_flag'] = data[col].isna().astype(int)
data = pd.concat([data, df_missing_flag], axis=1)

In [245]:
data['Repo Process Started Date'] = data['Repo Process Started Date'].map(lambda x: 0 if pd.isna(x) else 1)

# days with car when incident happend
data['Actual Pickup Date (ET)'] = pd.to_datetime(data['Actual Pickup Date (ET)'])
data['first_multiparty'] = pd.to_datetime(data['first_multiparty'])
data['first_multiparty'].fillna(pd.to_datetime("today"), inplace=True)
data['first_singleparty'] = pd.to_datetime(data['first_singleparty'])
data['first_singleparty'].fillna(pd.to_datetime("today"), inplace=True)
days_diff_mp = data['first_multiparty'] - data['Actual Pickup Date (ET)']
days_diff_sp = data['first_singleparty'] - data['Actual Pickup Date (ET)']
data['days_in_car_when_MP'] = days_diff_mp.dt.days
data['days_in_car_when_SP'] = days_diff_sp.dt.days
data = data[(data['days_in_car_when_MP']>0) & (data['days_in_car_when_SP']>0)] # remove columns if the days with car is negative


In [246]:
columns_to_drop = ['Single Party Incidents', 'Multi Party Incidents', 'early_selfchurn_customer_no_accdient', 
                          'seasoning_3month_SinglePartyIncident', 'seasoning_3month_MultiPartyIncident',
                          'Legacy Order Id', 'Most Recent Vehicle Id', 'Order ID', 'Subscriber ID',
                          'Actual Pickup Date (ET)', 'Returned Date (ET)', 'Cancellation Date (ET)', 'Creation Date (ET)', 'Scheduled Pickup Date (ET)',
                          'Repo Process Started Date',
                          'first_singleparty', 'first_multiparty', 
                          'Single Party Incidents_missing_flag', 'Multi Party Incidents_missing_flag',
                          'first_multiparty_missing_flag', 'first_singleparty_missing_flag'
                          ]

# missing_ratio = data.isnull().mean() 
# col_to_drop_m = missing_ratio[missing_ratio>0.2].index


In [247]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# data = pd.read_csv('../data/incidents.csv', low_memory=False)

from sklearn.compose import make_column_selector as selector


Target = 'incident_Ind'
y = data[Target]
X = data.drop(columns=[Target]+columns_to_drop, axis=1)

X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 9720 entries, 4 to 9739
Data columns (total 86 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Credit Score Bucket v2                                   9720 non-null   object 
 1   Age Bucket v2                                            9720 non-null   object 
 2   Market Name                                              9720 non-null   object 
 3   Days With Vehicle                                        9720 non-null   float64
 4   Miles Driven                                             9720 non-null   float64
 5   Is Cancelled                                             9720 non-null   object 
 6   Has at Least One Monthly Invoice                         9720 non-null   bool   
 7   Initial Invoice Is Monthly Invoice                       9720 non-null   bool   
 8   Pickup Location Name             

In [248]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X_train)
categorical_columns = categorical_columns_selector(X_train)


# preprocesser
numeric_transformer = Pipeline(
    steps=[("imputer_num", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("imputer_cat", SimpleImputer(strategy="most_frequent")), ("encode", OrdinalEncoder())]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, list(numerical_columns)),
        ("cat", categorical_transformer, list(categorical_columns)),
    ]
)

steps = [('preprocess', preprocessor),
         ('rf', RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=20, min_samples_leaf=1, min_samples_split=2))]

model = Pipeline(steps)
model_rf = model.fit(X_train, y_train)

In [249]:
feature_importances = pd.DataFrame(model_rf[1].feature_importances_, index = model_rf[:-1].get_feature_names_out())
# feature_importances[0].sort_values().plot(kind = 'barh', title = 'Random Forest')

In [250]:
feature_importances.sort_values(by=0, 
                                ascending=False).head(20)

,0
num__days_in_car_when_MP,0.378304
num__days_in_car_when_MP_missing_flag,0.177535
num__days_in_car_when_SP,0.109533
num__days_in_car_when_SP_missing_flag,0.056441
num__Days With Vehicle,0.027074
num__Total Collected Revenue,0.019363
num__Total Collected Vehicle Amount,0.016058
num__Total Revenue,0.012635
num__Miles Driven,0.010989
num__Total Revenue New,0.009624


In [240]:
import shap
explainer = shap.TreeExplainer(model_rf)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values[:,:,1], X)

InvalidModelError: Model type not yet supported by TreeExplainer: <class 'sklearn.pipeline.Pipeline'>

In [239]:
# from sklearn.metrics import roc_auc_score
# print('---------- Random Forest model -------------')
# print('Training data:', roc_auc_score(y_train, model_rf.predict_proba(X_train)[:, 1]))
# print('Test data:', roc_auc_score(y_test, model_rf.predict_proba(X_test)[:, 1]))